In [19]:
from typing import Union
import numpy as np
import random
import math


def randint(start: int,
            stop: int,
            step: int = 1,
            length: int = 0,
            dtype=object) -> Union[int, np.ndarray]:
    """
    Generate an array of random integers from [start, stop).
    :param start: start of the sample space (inclusive)
    :param stop: end of the sample space (exclusive)
    :param step: sample space includes every "step" number starting form "stop"
    :param length: length of the output array. If 0, an int is returned.
    :param dtype: dtype of the output array
    :return: 1D numpy array of random integers from [start, stop).
    """
    if length == 0:
        return random.randrange(start, stop, step)
    else:
        return np.array([random.randrange(start, stop, step) for _ in range(length)], dtype=dtype)


def int2binary_array(integer: int) -> np.ndarray:
    """
    Create a 1D array containing the binary digits of a non-negative integer.
    :param integer: non-negative integer of which the binary digits are to store in a 1D array.
    :return: 1D array of binary digits.
    """
    assert 0 <= integer
    return np.array(list(map(int, bin(integer)[2:])))


def binary_array2int(bits) -> int:
    """
    Create an integer from an array containing its binary digits.
    :param bits: 1D array of binary digits.
    :return: integer "bits" represents.
    """
    return int(''.join(map(str, bits)), 2)


def key_multiple(num_bits: int,
                 length: int) -> Union[int, np.ndarray]:
    """
    Sample q from [0, 2 ** num_bits), where q is intended for the encryption c = qp + 2r + m.
    :param num_bits: number of bits of q.
    :param length: length of output
    :return: an integer if length = 0, a 1D numpy array of shape (length,) if length > 0.
    """
    return randint(0, 2 ** num_bits, length=length)


def noise(key: int,
          length: int) -> Union[int, np.ndarray]:
    """
    Sample positive noise r, where r is intended for the encryption c = qp + 2r + m. The magnitude of r ensures
    homomorphism of addition or multiplication of ciphertexts for at least once.
    :param key: encryption key, i.e. p in c = qp + 2r + m
    :param length: length of output
    :return: an integer if length = 0, a 1D numpy array of shape (length,) if length > 0.
    """
    return randint(0, math.floor((-1 + math.sqrt(key - 1)) / 2), length=length)


def key(num_bits: int) -> int:
    """
    Generate an odd num_bits-bit integer from the range [2^{num_bits - 1}, 2^{num_bits}) as private key.
    :param num_bits: number of bits of the private key.
    :return: an odd integer private key
    """
    return randint(2 ** (num_bits - 1) + 1, 2 ** num_bits, step=2)


def encrypt(plaintext: int,
            key: int,
            q_num_bits: int) -> np.ndarray:
    """
    Encrypt a plaintext as c = qp + 2r + m, where m is a bit in the plaintext, p is a key, q is a factor of p,
    r is a noise, and c, an integer, is an encryption of m.
    :param plaintext: an integer to encrypt
    :param key: an encryption key
    :param q_num_bits: number of bits of q
    :return: 1D numpy array of encryption of each bit of plaintext
    """
    plaintext_binary = int2binary_array(plaintext)

    return (key_multiple(q_num_bits, len(plaintext_binary)) * key
            + 2 * noise(key, len(plaintext_binary))
            + plaintext_binary)


def decrypt(ciphertext: np.ndarray,
            key: int) -> int:
    """
    Decrypt.
    :param ciphertext: 1D numpy array of encryption of each bit of plaintext
    :param key: encryption key
    :return: plaintext as an integer
    """
    plaintext_binary = ciphertext % key % 2
    return binary_array2int(plaintext_binary)


## Settings

For simplicity assumes plaintexts have a fixed number of bits.

Please feel free to change the settings below, just make sure they are not too small.

In [20]:
key_num_bits = 5
factor_num_bits = 5
plaintext_num_bits = 5

## Generate Random Key and Plaintexts

Key is a random odd integer from $[2^{key\_num\_bits - 1}, 2^{key\_num\_bits})$

Plaintexts are random integers from $[2^{plaintext\_num\_bits - 1}, 2^{plaintext\_num\_bits})$

In [21]:
k = key(key_num_bits)
p0 = random.randint(2 ** (plaintext_num_bits - 1), 2 ** plaintext_num_bits)
p1 = random.randint(2 ** (plaintext_num_bits - 1), 2 ** plaintext_num_bits) 

print(f"Key is:\n"
      f"\t base 10: {k}\n"
      f"\t base 2: {bin(k)[2:]}\n")

print(f"Plaintext p0:\n"
      f"\t base 10: {p0}\n"
      f"\t base 2: {bin(p0)[2:]}\n"
      f"Plaintext p1:\n"
      f"\t base 10: {p1}\n"
      f"\t base 2: {bin(p1)[2:]}\n")

Key is:
	 base 10: 25
	 base 2: 11001

Plaintext p0:
	 base 10: 23
	 base 2: 10111
Plaintext p1:
	 base 10: 18
	 base 2: 10010


## Encryption

$$
ciphertext = factor \times key + 2 \times noise + plaintext\_bit
$$

In [22]:
c0 = encrypt(p0, k, factor_num_bits)
c1 = encrypt(p1, k, factor_num_bits)

print(f"Ciphertext c0:\n"
      f"\t base 10: {c0}\n"
      f"Ciphertext c1:\n"
      f"\t base 10: {c1}\n")

Ciphertext c0:
	 base 10: [576 300 326 426 251]
Ciphertext c1:
	 base 10: [426 375 325 151 425]


## Computation in Ciphertext Space

$$
D(E(p_0) + E(p_1)) = p_0 \oplus p_1 \\
D(E(p_0) \times E(p_1)) = p_0 \wedge p_1
$$



In [23]:
c_sum = c0 + c1
c_product = c0 * c1

print(f"c0 + c1:\n"
      f"\t base 10: {c_sum}\n"
      f"c0 * c1:\n"
      f"\t base 10: {c_product}\n")

c0 + c1:
	 base 10: [1002 675 651 577 676]
c0 * c1:
	 base 10: [245376 112500 105950 64326 106675]


## Decryption

In [24]:
decrypted_c0 = decrypt(c0, k)
decrypted_c1 = decrypt(c1, k)
decrypted_sum = decrypt(c_sum, k)
decrypted_product = decrypt(c_product, k)

print(f"Original plaintexts:\n"
      f"\tp0 = {bin(p0)[2:]}\n"
      f"\tp1 = {bin(p1)[2:]}:\n"
      f"Decryption results:\n"
      f"\tDecrypt(c0) = {bin(decrypted_c0)[2:]}\n"
      f"\tDecrypt(c1) = {bin(decrypted_c1)[2:]}\n"
      f"\tDecrypt(c0 + c1) = {(bin(decrypted_sum)[2:]).zfill(plaintext_num_bits)} = p0 xor p1\n"
      f"\tDecrypt(c0 * c1) = {(bin(decrypted_product)[2:]).zfill(plaintext_num_bits)} = p0 and p1")

Original plaintexts:
	p0 = 10111
	p1 = 10010:
Decryption results:
	Decrypt(c0) = 10111
	Decrypt(c1) = 10010
	Decrypt(c0 + c1) = 00101 = p0 xor p1
	Decrypt(c0 * c1) = 10010 = p0 and p1
